In [1]:
import numpy as np
import pandas as pd

In [2]:
cases = pd.read_csv("data100k.csv")

In [4]:
cases.head(3).T

,0,1,2
person_id,102090000000110,343221000000125,343221000000125
HearingDate,2019-02-28,2009-12-07,2011-01-20
CodeSection,A.46.2-862,B.46.2-301,A.46.2-707
codesection,covered elsewhere,covered elsewhere,covered elsewhere
ChargeType,Misdemeanor,Misdemeanor,Misdemeanor
chargetype,Misdemeanor,Misdemeanor,Misdemeanor
Class,1,1,3
DispositionCode,Guilty,Guilty,Guilty
disposition,Conviction,Conviction,Conviction
Plea,NaN,NaN,NaN


## What code sections are most frequent?


In [6]:
cases['CodeSection'].value_counts()

CodeSection
A.46.2-862        26379
B.46.2-301        25967
46.2-300          17934
C.46.2-862        11728
18.2-250.1        10573
                  ...  
18.2-135              1
18.2-181; -104        1
26-29                 1
26.27                 1
4.1-30B               1
Name: count, Length: 4207, dtype: int64

## Which ones most often lead to convictions?


In [7]:
cases['DispositionCode'].value_counts()

DispositionCode
Guilty                     156563
Nolle Prosequi              54680
Dismissed                   42520
Guilty In Absentia          31958
Not Guilty                   5807
Not Guilty/Acquitted         1623
Not True Bill                 250
No Indictment Presented       178
Dismissed/Other                19
Name: count, dtype: int64

In [10]:
cases['convicted'] = [x in ['Guilty', 'Guilty In Absentia'] for x in cases['DispositionCode']]

In [11]:
cases['convicted'].value_counts()

convicted
True     188521
False    105077
Name: count, dtype: int64

In [19]:
cases_convict_rate = cases.groupby('CodeSection').agg({'convicted': ['mean', 'count']}).reset_index()
cases_convict_rate.columns = ['CodeSection', 'convict_rate', 'count']
cases_convict_rate = cases_convict_rate.query("count >= 30")
cases_convict_rate.sort_values('convict_rate', ascending=False)

,CodeSection,convict_rate,count
1806,23-55,0.981818,55
1633,21-336,0.960000,50
1755,23-22.1(A),0.954198,131
2103,29-17(C),0.942857,70
4111,G.18.2-266,0.930233,43
...,...,...,...
2321,3.2-6503.1,0.071429,42
140,11.1-2,0.052632,38
2481,35-416,0.033333,30
1433,19.2-100,0.000000,238


## Which ones have the most severe racial disparities?


In [22]:
cases['Race'].unique()

array(['Black(Non-Hispanic)', 'Hispanic', 'White Caucasian(Non-Hispanic)',
       'MISSING', 'Asian Or Pacific Islander', 'Black (Non-Hispanic)',
       'White Caucasian (Non-Hispanic)',
       'Other(Includes Not Applicable.. Unknown)',
       'Other (Includes Not Applicable.. Unknown)', 'Black', 'White',
       'Unknown (Includes Not Applicable.. Unknown)', 'American Indian',
       'Unknown', 'Asian or Pacific Islander',
       'American Indian Or Alaskan Native'], dtype=object)

In [23]:
replace_map = {
        'Black(Non-Hispanic)' : 'Black (Non-Hispanic)', 
        'Hispanic' : 'Hispanic',
        'White Caucasian(Non-Hispanic)': 'White (Non-Hispanic)',
        'MISSING': "Other, Unknown, or Missing",
        'Asian Or Pacific Islander' : 'Asian or Pacific Islander',
        'Black (Non-Hispanic)' : 'Black (Non-Hispanic)',
        'White Caucasian (Non-Hispanic)': 'White (Non-Hispanic)', 
        'Other(Includes Not Applicable.. Unknown)': "Other, Unknown, or Missing",
        'Other (Includes Not Applicable.. Unknown)': "Other, Unknown, or Missing",
        'Black': 'Black (Non-Hispanic)', 
        'White': 'White (Non-Hispanic)',
        'Unknown (Includes Not Applicable.. Unknown)': "Other, Unknown, or Missing", 
        'American Indian': 'American Indian or Alaskan Native',
        'Unknown': "Other, Unknown, or Missing", 
        'Asian or Pacific Islander' : 'Asian or Pacific Islander',
        'American Indian Or Alaskan Native': 'American Indian or Alaskan Native'}
cases['Race'] = cases['Race'].replace(replace_map)
cases['Race'].value_counts()

Race
White (Non-Hispanic)                 159627
Black (Non-Hispanic)                 115627
Hispanic                               9319
Other, Unknown, or Missing             5928
Asian or Pacific Islander              2794
American Indian or Alaskan Native       303
Name: count, dtype: int64

## In what localities (fips) are these disparities most severe?